### Estimator API

In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tensorflow import keras
import pandas as pd
import os

In [2]:
train_file = "./data/titanic/train.csv"
eval_file = "./data/titanic/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

# dataframe pip function remove column from dataset and take the removed column
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

print(train_df.head())
print(eval_df.head())

      sex   age  n_siblings_spouses  parch  ...  class     deck  embark_town alone
0    male  22.0                   1      0  ...  Third  unknown  Southampton     n
1  female  38.0                   1      0  ...  First        C    Cherbourg     n
2  female  26.0                   0      0  ...  Third  unknown  Southampton     y
3  female  35.0                   1      0  ...  First        C  Southampton     n
4    male  28.0                   0      0  ...  Third  unknown   Queenstown     y

[5 rows x 9 columns]
      sex   age  n_siblings_spouses  parch  ...   class     deck  embark_town alone
0    male  35.0                   0      0  ...   Third  unknown  Southampton     y
1    male  54.0                   0      0  ...   First        E  Southampton     y
2  female  58.0                   0      0  ...   First        C  Southampton     y
3  female  55.0                   0      0  ...  Second  unknown  Southampton     y
4    male  34.0                   0      0  ...  Second     

In [3]:
# discrete value
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class',
                      'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

feature_columns = []

for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    
    print(categorical_column, vocab)
    
    feature_columns.append(
        # indicator_column -> one_hot discrete value
        tf.feature_column.indicator_column(
            # fill feature_column with vocab data (name, possible value)
            tf.feature_column.categorical_column_with_vocabulary_list(
                categorical_column,
                vocab)))

for numeric_column in numeric_columns:    
    feature_columns.append(
        # for numeric value just add them directly to feature_column
        tf.feature_column.numeric_column(numeric_column, dtype=tf.float32)
    )

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [4]:
def make_dataset(data_df, label_df, epochs = 10, shuffle = True,
                batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(data_df), label_df))
    
    if shuffle == True:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [5]:
model = keras.models.Sequential([
    keras.layers.DenseFeatures(feature_columns=feature_columns),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(2, activation = 'softmax')
])

model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(learning_rate=0.01),
              metrics = ["accuracy"])

In [6]:
output_dir = 'baseline_model'

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

baseline_estimator = tf.estimator.BaselineClassifier(
    model_dir = output_dir,
    n_classes = 2
)

baseline_estimator.train(input_fn = lambda: make_dataset(
    train_df, y_train, epochs = 100
))

W0724 10:00:28.642052 140406365710144 deprecation.py:323] From /home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0724 10:00:28.773476 140406365710144 deprecation.py:323] From /home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/head/base_head.py:574: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0724 10:00:28.845140 140406365710144 deprecation.py:323] From /home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensor

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext (defined at <ipython-input-6-12ed84233911>:11) ]]

Original stack trace for 'IteratorGetNext':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/spyder_kernels/console/__main__.py", line 11, in <module>
    start.main()
  File "/home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/spyder_kernels/console/start.py", line 318, in main
    kernel.start()
  File "/home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/torna

In [7]:
baseline_estimator.evaluate(input_fn = lambda: make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False, batch_size = 20))

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext (defined at <ipython-input-7-2e84ee3fa94d>:1) ]]

Original stack trace for 'IteratorGetNext':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/spyder_kernels/console/__main__.py", line 11, in <module>
    start.main()
  File "/home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/spyder_kernels/console/start.py", line 318, in main
    kernel.start()
  File "/home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/bai/.virtualenvs/tensorflow2/lib/python3.6/site-packages/tornad

In [ ]:
# 2. use estimator (alternative to fit)
estimator = keras.estimator.model_to_estimator(model)

# input_fn -> must be a function i.g. a lambda function
# return 
# a. (features, labels)
# b. dataset(feature, label)
estimator.train(
    input_fn = lambda: make_dataset(train_df, y_train, epochs = 100)
)